# Vertex SDK: BigQuery Custom Container Training Example

# Install Vertex SDK


After the SDK installation the kernel will be automatically restarted.

In [ ]:
!pip3 uninstall -y google-cloud-aiplatform
!pip3 install google-cloud-aiplatform
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

### Enter your project and GCS bucket

Enter your Project Id in the cell below. Then run the cell to make sure the Cloud SDK uses the right project for all the commands in this notebook.

In [1]:
MY_PROJECT = "crazy-hippo-01"
MY_STAGING_BUCKET = "gs://crazy-vertex-ai-pipelines"  # bucket should be in same region as ucaip

# Copy Big Query Iris Dataset
We will make a Big Query dataset and copy Big Query's public iris table to that dataset. For more information about this dataset please visit: https://archive.ics.uci.edu/ml/datasets/iris 

### Make BQ Dataset

In [2]:
import os

os.environ['MY_PROJECT'] = MY_PROJECT
!bq mk {MY_PROJECT}:ml_iris

Dataset 'crazy-hippo-01:ml_iris' successfully created.


### Copy bigquery-public-data.ml_datasets.iris

In [3]:
!bq cp -n bigquery-public-data:ml_datasets.iris {MY_PROJECT}:ml_iris.iris_raw

Waiting on bqjob_r72dbc8e433bbd436_00000179a2c5512d_1 ... (0s) Current status: DONE   
Table 'bigquery-public-data:ml_datasets.iris' successfully copied to 'crazy-hippo-01:ml_iris.iris_raw'


# Create Training Container
We will create a directory and write all of our container build artifacts into that folder.

In [4]:
CONTAINER_ARTIFACTS_DIR = "demo-container-artifacts"

!mkdir {CONTAINER_ARTIFACTS_DIR}

### Create Cloudbuild YAML

In [5]:
cloudbuild_yaml = """steps:
- name: 'gcr.io/cloud-builders/docker'
  args: [ 'build', '-t', 'gcr.io/{MY_PROJECT}/test-custom-container', '.' ]
images: ['gcr.io/{MY_PROJECT}/test-custom-container']""".format(
    MY_PROJECT=MY_PROJECT
)

with open(f"{CONTAINER_ARTIFACTS_DIR}/cloudbuild.yaml", "w") as fp:
    fp.write(cloudbuild_yaml)

### Write Dockerfile

In [6]:
%%writefile {CONTAINER_ARTIFACTS_DIR}/Dockerfile

# Specifies base image and tag
FROM gcr.io/google-appengine/python
WORKDIR /root

# Installs additional packages
RUN pip3 install tensorflow tensorflow-io pyarrow

# Copies the trainer code to the docker image.
COPY test_script.py /root/test_script.py

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python3", "test_script.py"]

Writing demo-container-artifacts/Dockerfile


### Write entrypoint script to invoke trainer

In [7]:
%%writefile {CONTAINER_ARTIFACTS_DIR}/test_script.py

from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from tensorflow_io.bigquery import BigQueryClient
from tensorflow_io.bigquery import BigQueryReadSession
import tensorflow as tf
from tensorflow import feature_column
import os

training_data_uri = os.environ["AIP_TRAINING_DATA_URI"]
validation_data_uri = os.environ["AIP_VALIDATION_DATA_URI"]
test_data_uri = os.environ["AIP_TEST_DATA_URI"]
data_format = os.environ["AIP_DATA_FORMAT"]

def caip_uri_to_fields(uri):
    uri = uri[5:]
    project, dataset, table = uri.split('.')
    return project, dataset, table

feature_names = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

target_name = 'species'

def transform_row(row_dict):
  # Trim all string tensors
  trimmed_dict = { column:
                  (tf.strings.strip(tensor) if tensor.dtype == 'string' else tensor) 
                  for (column,tensor) in row_dict.items()
                  }
  target = trimmed_dict.pop(target_name)

  target_float = tf.cond(tf.equal(tf.strings.strip(target), 'versicolor'), 
                 lambda: tf.constant(1.0),
                 lambda: tf.constant(0.0))
  return (trimmed_dict, target_float)

def read_bigquery(project, dataset, table):
  tensorflow_io_bigquery_client = BigQueryClient()
  read_session = tensorflow_io_bigquery_client.read_session(
      "projects/" + project,
      project, table, dataset,
      feature_names + [target_name],
      [dtypes.float64] * 4 + [dtypes.string],
      requested_streams=2)

  dataset = read_session.parallel_read_rows()
  transformed_ds = dataset.map(transform_row)
  return transformed_ds

BATCH_SIZE = 16

training_ds = read_bigquery(*caip_uri_to_fields(training_data_uri)).shuffle(10).batch(BATCH_SIZE)
eval_ds = read_bigquery(*caip_uri_to_fields(validation_data_uri)).batch(BATCH_SIZE)
test_ds = read_bigquery(*caip_uri_to_fields(test_data_uri)).batch(BATCH_SIZE)

feature_columns = []

# numeric cols
for header in feature_names:
  feature_columns.append(feature_column.numeric_column(header))

feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

Dense = tf.keras.layers.Dense
model = tf.keras.Sequential(
  [
    feature_layer,
      Dense(16, activation=tf.nn.relu),
      Dense(8, activation=tf.nn.relu),
      Dense(4, activation=tf.nn.relu),
      Dense(1, activation=tf.nn.sigmoid),
  ])

# Compile Keras model
model.compile(
    loss='binary_crossentropy', 
    metrics=['accuracy'],
    optimizer='adam')

model.fit(training_ds, epochs=5, validation_data=eval_ds)

print(model.evaluate(test_ds))

tf.saved_model.save(model, os.environ["AIP_MODEL_DIR"])

Writing demo-container-artifacts/test_script.py


### Build Container

In [8]:
!gcloud builds submit --config {CONTAINER_ARTIFACTS_DIR}/cloudbuild.yaml {CONTAINER_ARTIFACTS_DIR}

Creating temporary tarball archive of 3 file(s) totalling 3.1 KiB before compression.
Uploading tarball of [demo-container-artifacts] to [gs://crazy-hippo-01_cloudbuild/source/1621933595.237262-0cb10e9fe2dc4248bb61c8cb3030a612.tgz]
Created [https://cloudbuild.googleapis.com/v1/projects/crazy-hippo-01/locations/global/builds/a373a1e5-cb09-4224-9304-d2f2703449ab].
Logs are available at [https://console.cloud.google.com/cloud-build/builds/a373a1e5-cb09-4224-9304-d2f2703449ab?project=433654631026].
----------------------------- REMOTE BUILD OUTPUT ------------------------------
starting build "a373a1e5-cb09-4224-9304-d2f2703449ab"

FETCHSOURCE
Fetching storage object: gs://crazy-hippo-01_cloudbuild/source/1621933595.237262-0cb10e9fe2dc4248bb61c8cb3030a612.tgz#1621934315511773
Copying gs://crazy-hippo-01_cloudbuild/source/1621933595.237262-0cb10e9fe2dc4248bb61c8cb3030a612.tgz#1621934315511773...
/ [1 files][  1.4 KiB/  1.4 KiB]                                                
Operation compl

# Run Custom Container Training

## Initialize Vertex AI SDK

Initialize the *client* for Vertex AI

In [17]:
from google.cloud import aiplatform
MY_PROJECT = "crazy-hippo-01"

aiplatform.init(project=MY_PROJECT, staging_bucket=MY_STAGING_BUCKET)

# Create a Managed Tabular Dataset from Big Query Dataset

This section will create a managed Tabular dataset from the iris Big Query table we copied above.

In [10]:
ds = aiplatform.TabularDataset.create(
    display_name="bq_iris_dataset", bq_source=f"bq://{MY_PROJECT}.ml_iris.iris_raw"
)

INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/433654631026/locations/us-central1/datasets/7915155509940846592/operations/2700835414666641408
INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/433654631026/locations/us-central1/datasets/7915155509940846592
INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/433654631026/locations/us-central1/datasets/7915155509940846592')


# Launch a Training Job to Create a Model

We will train a model with the container we built above.

In [11]:
job = aiplatform.CustomContainerTrainingJob(
    display_name="train-bq-iris",
    container_uri=f"gcr.io/{MY_PROJECT}/test-custom-container:latest",
    model_serving_container_image_uri="gcr.io/cloud-aiplatform/prediction/tf2-cpu.2-2:latest",
)
model = job.run(
    ds,
    replica_count=1,
    model_display_name="bq-iris-model",
    bigquery_destination=f"bq://{MY_PROJECT}",
)

INFO:google.cloud.aiplatform.training_jobs:Training Output directory:
gs://crazy-vertex-ai-pipelines/aiplatform-custom-training-2021-05-25-09:15:06.583 
INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/7244312679509131264?project=433654631026
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTrainingJob projects/433654631026/locations/us-central1/trainingPipelines/7244312679509131264 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTrainingJob projects/433654631026/locations/us-central1/trainingPipelines/7244312679509131264 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTrainingJob projects/433654631026/locations/us-central1/trainingPipelines/7244312679509131264 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:CustomContainerTr

# Deploy Your Model

Deploy your model, then wait until the model FINISHES deployment before proceeding to prediction.

In [12]:
endpoint = model.deploy(machine_type="n1-standard-4")

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/433654631026/locations/us-central1/endpoints/8919431838565400576/operations/5156423111490404352
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/433654631026/locations/us-central1/endpoints/8919431838565400576
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/433654631026/locations/us-central1/endpoints/8919431838565400576')
INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/433654631026/locations/us-central1/endpoints/8919431838565400576
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/433654631026/locations/us-central1/endpoints/8919431838565400576/operations/3805343223279255552
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/43365463102

In [23]:
aiplatform.Endpoint.list()

 resource name: projects/433654631026/locations/us-central1/endpoints/8919431838565400576,
 resource name: projects/433654631026/locations/us-central1/endpoints/6696113774086062080,
 resource name: projects/433654631026/locations/us-central1/endpoints/2987645771551080448]

# Predict on the Endpoint

#### Prediction is a binary classification for <b>if the flower is a versicolor iris flower of not</b>. 

In [29]:
endpoint.predict(
    [{"sepal_length": 4.1, "sepal_width": 2.5, "petal_length": 1.0, "petal_width": 1.1}]
)

Prediction(predictions=[[0.334066838]], deployed_model_id='4748957846131965952', explanations=None)